In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc,  font_manager
import matplotlib
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import statistics
matplotlib.rcParams['axes.unicode_minus'] = False
fontname = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=fontname).get_name()
rc('font', family=font_name)

In [ ]:
get_data = pd.read_csv('../data/TEMP/경락가격/당근/당근 월별 총거래(정제).csv', index_col=None)
# year_df = pd.read_csv('../data/TEMP/경락가격/오이/오이 년별 평균가격.csv', index_col=None)
# get_data = pd.read_csv('../data/TEMP/경락가격/오이/오이 월별 총거래(정제2).csv', index_col=None)

In [ ]:
get_data['Date'] = pd.to_datetime(get_data['Date'])

In [ ]:
years = get_data['Date'].dt.year.unique()

In [ ]:

year_mean = []
year_addon = []
for i in years:
    for j in get_data.columns:
        if '평균' in j:
            print(get_data.loc[get_data['Date'].dt.year == i,'%s' %j][:5])
            data_p = get_data.loc[get_data['Date'].dt.year == i,'%s' %j].values
            year_addon.append(data_p.sum())
            # np_std= np.std(get_data.loc[get_data['Date'].dt.year == i,'%s' %j].values)
            # print('표준편차: ',np_std)
            # year_addon.append(np_std)
        elif j != 'Date':
            data_p = get_data.loc[get_data['Date'].dt.year == i,'%s' %j].values
            year_addon.append(data_p.sum())
        else:
            year_addon.append(i)
    year_mean.append(year_addon)
    year_addon = []

year_mean

In [ ]:
year_df = pd.DataFrame(year_mean, columns=['year', '서울 총거래물량', '서울 총거래금액', '서울 평균단가','부산 총거래물량', '부산 총거래금액', '부산 평균단가', '대구 총거래물량', '대구 총거래금액', '대구 평균단가', '대전 총거래물량', '대전 총거래금액', '대전 평균단가', '광주 총거래물량', '광주 총거래금액', '광주 평균단가'])

In [ ]:
for i in years:
    year_df.loc[year_df['year']== i,'서울 평균단가'] = year_df['서울 총거래금액']/year_df['서울 총거래물량']
    year_df.loc[year_df['year']== i,'부산 평균단가'] = year_df['부산 총거래금액']/year_df['부산 총거래물량']
    year_df.loc[year_df['year']== i,'대구 평균단가'] = year_df['대구 총거래금액']/year_df['대구 총거래물량']
    year_df.loc[year_df['year']== i,'대전 평균단가'] = year_df['대전 총거래금액']/year_df['대전 총거래물량']
    year_df.loc[year_df['year']== i,'광주 평균단가'] = year_df['광주 총거래금액']/year_df['광주 총거래물량']

year_df

In [ ]:
get_data = get_data.rename(columns={'서울 평균거래금액':'서울 평균단가', '부산 평균거래금액':'부산 평균단가', '대구 평균거래금액':'대구 평균단가', '대전 평균거래금액':'대전 평균단가', '광주 평균거래금액':'광주 평균단가'})

In [ ]:
year_df['전국 평균단가'] = (year_df['서울 총거래금액'] + year_df['부산 총거래금액'] + year_df['대구 총거래금액'] + year_df['대전 총거래금액'] + year_df['광주 총거래금액']) / (year_df['서울 총거래물량'] + year_df['부산 총거래물량'] + year_df['대구 총거래물량'] + year_df['대전 총거래물량'] + year_df['광주 총거래물량']) 

In [ ]:
year_df.to_csv('../data/TEMP/경락가격/당근/당근 년별 평균가격.csv', index=False, encoding='UTF-8')

In [ ]:
year_df

In [ ]:
years = get_data['Date'].dt.year.unique()
months = get_data['Date'].dt.month.unique()
grade_data = []
for i in years:
    for j in months:
        if ( i != 2023 ) or ( j not in [9, 10, 11, 12]):
            g_date = str(i)+"-"+str(j)
            seoul_data = year_df.loc[year_df['year']==i, '서울 평균단가'].values
            seoul_data1 = get_data.loc[(get_data['Date'].dt.month == j) & (get_data['Date'].dt.year==i), '서울 평균단가'].values
            seoul_grade = seoul_data - seoul_data1

            busan_data = year_df.loc[year_df['year']==i, '부산 평균단가'].values
            busan_data1 = get_data.loc[(get_data['Date'].dt.month == j) & (get_data['Date'].dt.year==i), '부산 평균단가'].values
            busan_grade = busan_data - busan_data1

            daegu_data = year_df.loc[year_df['year']==i, '대구 평균단가'].values
            daegu_data1 = get_data.loc[(get_data['Date'].dt.month == j) & (get_data['Date'].dt.year==i), '대구 평균단가'].values
            daegu_grade = daegu_data - daegu_data1

            daejeon_data = year_df.loc[year_df['year']==i, '대전 평균단가'].values
            daejeon_data1 = get_data.loc[(get_data['Date'].dt.month == j) & (get_data['Date'].dt.year==i), '광주 평균단가'].values
            daejeon_grade = daejeon_data - daejeon_data1

            gwangju_data = year_df.loc[year_df['year']==i, '광주 평균단가'].values
            gwangju_data1 = get_data.loc[(get_data['Date'].dt.month == j) & (get_data['Date'].dt.year==i), '광주 평균단가'].values
            gwangju_grade = gwangju_data - gwangju_data1
            print(len(gwangju_grade))
            grade_data.append([g_date,seoul_grade[0], busan_grade[0], daegu_grade[0], daejeon_grade[0], gwangju_grade[0]])
print(grade_data)

In [ ]:
mean_data = pd.DataFrame(grade_data, columns=['Date','서울 평균단가 편차', '부산 평균단가 편차', '대구 평균단가 편차', '대전 평균단가 편차', '광주 평균단가 편차'])

In [ ]:
mean_data['Date'] = pd.to_datetime(mean_data['Date'])

In [ ]:
mean_all = pd.merge(get_data, mean_data, on='Date', how='inner')

In [ ]:
mean_all = mean_all[['Date', '서울 총거래물량', '서울 총거래금액', '서울 평균단가', '서울 평균단가 편차','부산 총거래물량', '부산 총거래금액', '부산 평균단가', '부산 평균단가 편차', '대구 총거래물량', '대구 총거래금액', '대구 평균단가', '대구 평균단가 편차','대전 총거래물량', '대전 총거래금액', '대전 평균단가', '대전 평균단가 편차', '광주 총거래물량', '광주 총거래금액', '광주 평균단가', '광주 평균단가 편차']]

In [ ]:
mean_all['전국 평균단가'] = (mean_all['서울 총거래금액'] + mean_all['부산 총거래금액'] + mean_all['대구 총거래금액'] + mean_all['대전 총거래금액'] + mean_all['광주 총거래금액']) / (mean_all['서울 총거래물량'] + mean_all['부산 총거래물량'] + mean_all['대구 총거래물량'] + mean_all['대전 총거래물량'] + mean_all['광주 총거래물량']) 

In [ ]:
mean_all.to_csv('../data/TEMP/경락가격/당근/당근 월별 데이터와 평균편차.csv', index=False, encoding='UTF-8')